In [76]:
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display
import ipywidgets as widgets
import numpy as np

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
URLS = {
    "Opole": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/opolskie/opole/opole/opole?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Toruń": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/kujawsko--pomorskie/torun/torun?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Bydgoszcz": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/kujawsko--pomorskie/bydgoszcz/bydgoszcz?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Lublin": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/lubelskie/lublin/lublin?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Zielona Góra": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/lubuskie/zielona-gora/zielona-gora?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Łódz": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/lodzkie/lodz/lodz?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Opole": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/opolskie/opole/opole?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Rzeszów": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/podkarpackie/rzeszow/rzeszow?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Białystok": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/podlaskie/bialystok/bialystok?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Gdańsk": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/pomorskie/gdansk/gdansk?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Katowice": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/slaskie/katowice/katowice?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Kielce": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/swietokrzyskie/kielce/kielce?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Olsztyn": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/warminsko--mazurskie/olsztyn/olsztyn/olsztyn?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Poznań": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/wielkopolskie/poznan/poznan?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Szczecin": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/zachodniopomorskie/szczecin/szczecin?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Kraków": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/malopolskie/krakow/krakow?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
    "Warszawa": "https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/mazowieckie/warszawa/warszawa?limit=72&page={}&ownerTypeSingleSelect=ALL&buildYearMax=2025&by=DEFAULT&direction=DESC",
}


HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
MAX_PAGES = 500

pd.set_option('display.max_colwidth', None)

# -----------------------------------------------------------------------------
# Data references
# -----------------------------------------------------------------------------
# =============================================================================
# 🗺️ Polish Administrative Data — Regions, Cities & Districts
# =============================================================================
# This cell defines the mapping between Polish voivodeships (regions),
# their main cities, and the corresponding districts (dzielnice).
# It’s used for parsing and categorizing location strings from real-estate data.
# =============================================================================

# --- Voivodeships (regions) and their main cities ---
REGIONS = {
    "dolnośląskie": ["Wrocław"],
    "kujawsko-pomorskie": ["Bydgoszcz", "Toruń"],
    "lubelskie": ["Lublin"],
    "lubuskie": ["Zielona Góra", "Gorzów Wielkopolski"],
    "łódzkie": ["Łódź"],
    "małopolskie": ["Kraków"],
    "mazowieckie": ["Warszawa"],
    "opolskie": ["Opole"],
    "podkarpackie": ["Rzeszów"],
    "podlaskie": ["Białystok"],
    "pomorskie": ["Gdańsk"],
    "śląskie": ["Katowice"],
    "świętokrzyskie": ["Kielce"],
    "warmińsko-mazurskie": ["Olsztyn"],
    "wielkopolskie": ["Poznań"],
    "zachodniopomorskie": ["Szczecin"]
}

# --- Major cities and their districts ---
WAW_DISTRICTS = [
    "Bemowo", "Białołęka", "Bielany", "Mokotów", "Ochota",
    "Praga-Południe", "Praga-Północ", "Rembertów", "Śródmieście",
    "Targówek", "Ursus", "Ursynów", "Wawer", "Wesoła",
    "Wilanów", "Włochy", "Wola", "Żoliborz"
]

KRA_DISTRICTS = [
    "Stare Miasto", "Grzegórzki", "Prądnik Czerwony", "Prądnik Biały",
    "Krowodrza", "Bronowice", "Zwierzyniec", "Dębniki",
    "Łagiewniki-Borek Fałęcki", "Swoszowice", "Podgórze Duchackie",
    "Bieżanów-Prokocim", "Podgórze", "Czyżyny", "Mistrzejowice",
    "Bieńczyce", "Wzgórza Krzesławickie"
]

BIA_DISTRICTS = [
    "Centrum", "Białostoczek", "Sienkiewicza", "Bojary", "Piaski",
    "Antoniuk", "Jaroszówka", "Wygoda", "Piasta I i II", "Wysoki Stoczek",
    "Dziesięciny I i II", "Bacieczki", "Starosielce", "Dojlidy"
]

BYD_DISTRICTS = [
    "Babia Wieś", "Bartodzieje", "Bielawy", "Błonie", "Bocianowo-Śródmieście-Stare Miasto",
    "Brdyujście", "Bydgoszcz Wschód-Siernieczek", "Czyżkówko", "Flisy",
    "Glinki-Rupienica", "Górzyskowo", "Jachcice", "Kapuściska", "Leśne",
    "Łęgnowo", "Łęgnowo Wieś", "Miedzyń-Prądy", "Nowy Fordon", "Okole",
    "Osowa Góra", "Piaski", "Smukała-Opławiec-Janowo", "Stary Fordon",
    "Szwederowo", "Tatrzańskie", "Terenów Nadwiślańskich", "Wilczak-Jary",
    "Wyżyny", "Wzgórze Wolności", "Zimne Wody–Czersko Polskie"
]

GDA_DISTRICTS = [
    "Aniołki", "Brętowo", "Brzeźno", "Chełm", "Jasień", "Kokoszki",
    "Krakowiec-Górki Zachodnie", "Letnica", "Matarnia", "Młyniska",
    "Nowy Port", "Oliwa", "Olszynka", "Orunia-Św. Wojciech-Lipce",
    "Orunia Górna-Gdańsk Południe", "Osowa", "Piecki-Migowo", "Przeróbka",
    "Przymorze Małe", "Przymorze Wielkie", "Rudniki", "Siedlce", "Stogi",
    "Suchanino", "Śródmieście", "Ujeścisko-Łostowice", "Wrzeszcz Dolny",
    "Wrzeszcz Górny", "Zaspa-Młyniec", "Zaspa-Rozstaje",
    "Żabianka-Wejhera-Jelitkowo-Tysiąclecia"
]

GOR_DISTRICTS = [
    "Baczyna", "Chróścik", "Chwalęcice", "Górczyn", "Janice", "Karnin",
    "Małyszyn Wielki", "Małyszyn Mały", "Nowy Dwór", "Piaski", "Sady",
    "Śródmieście", "Zakanale"
]

KIE_DISTRICTS = [
    "Baranówek", "Barwinek", "Białogon", "Biesak", "Bocianek", "Bukówka",
    "Cedro-Mazur", "Cegielnia", "Centrum", "Chęcińskie", "Czarnów",
    "Dąbrowa", "Dobromyśl", "Domaszowice Wikaryjskie", "Dyminy-Wieś",
    "Głęboczka", "Herby", "Jagiellońskie", "Karczówka", "Łazy", "Malików",
    "Na Stoku", "Nowy Folwark", "Niewachlów I", "Niewachlów II",
    "Osiedle Jana Czarnockiego", "Osiedle Jana Kochanowskiego", "Ostra Górka",
    "Pakosz", "Panorama", "Piaski", "Pietraszki", "Pod Dalnią", "Podhale",
    "Podkarczówka", "Pod Telegrafem", "Posłowice", "Sady", "Sandomierskie",
    "Sieje", "Sitkówka", "Skrzetle", "Słoneczne Wzgórze", "Słowik",
    "Szydłówek", "Ślichowice", "Świętokrzyskie", "Uroczysko", "Wielkopole",
    "Wietrznia", "Zacisze", "Zalesie", "Zagórska Południe",
    "Zagórska Północ", "Zagórze", "Związkowiec"
]

KAT_DISTRICTS = [
    "Śródmieście", "Koszutka", "Bogucice", "Osiedle Paderewskiego – Muchowiec",
    "Załęże", "Osiedle Wincentego Witosa", "Osiedle Tysiąclecia", "Dąb",
    "Wełnowiec-Józefowiec", "Ligota-Panewniki", "Brynów-Osiedle Zgrzebnioka",
    "Załęska Hałda-Brynów", "Piotrowice-Ochojec", "Szopienice-Burowiec",
    "Murkowice", "Kostuchna", "Piotrowice", "Ochojec", "Zarzecze",
    "Dąbrówka Mała", "Stare Bogucice", "Nowe Bogucice"
]

LUB_DISTRICTS = [
    "Abramowice", "Bronowice", "Czechów Południowy", "Czechów Północny",
    "Czuby Południowe", "Czuby Północne", "Dziesiąta", "Felin", "Głusk",
    "Hajdów-Zadębie", "Kalinowszczyzna", "Konstantynów", "Kośminek",
    "Ponikwoda", "Rury", "Sławin", "Sławinek", "Stare Miasto", "Szerokie",
    "Śródmieście", "Tatary", "Węglin Południowy", "Węglin Północny",
    "Wieniawa", "Wrotków", "Za Cukrownią", "Zemborzyce"
]

LOD_DISTRICTS = ["Bałuty", "Górna", "Polesie", "Śródmieście", "Widzew"]

OLS_DISTRICTS = [
    "Brzeziny", "Dajtki", "Generałów", "Grunwaldzkie", "Gutkowo", "Jaroty",
    "Kętrzyńskiego", "Kormoran", "Kortowo", "Kościuszki", "Likusy",
    "Mazurskie", "Mleczna", "Nad Jeziorem Długim", "Nagórki", "Pieczewo",
    "Podgrodzie", "Podleśna", "Pojezierze", "Redykajny", "Śródmieście",
    "Wojska Polskiego", "Zatorze", "Zielona Górka"
]

OPO_DISTRICTS = [
    "Borki", "Brzezie", "Czarnowąsy", "Świerkle", "Krzanowice",
    "Wróblin", "Zakrzów", "Chabry", "Armii Krajowej", "Gosławice",
    "Malinka", "Nowa Wieś Królewska", "Półwieś"
]

POZ_DISTRICTS = [
    "Antoninek-Zieliniec-Kobylepole", "Chartowo", "Fabianowo-Kotowo",
    "Główna", "Głuszyna", "Górczyn", "Grunwald Północ", "Grunwald Południe",
    "Jeżyce", "Junikowo", "Kiekrz", "Krzesiny-Pokrzywno-Garaszewo",
    "Krzyżowniki-Smochowice", "Kwiatowe", "Ławica", "Morasko-Radojewo",
    "Naramowice", "Nowe Winogrady Południe", "Nowe Winogrady Północ",
    "Nowe Winogrady Wschód", "Ogrody", "Ostrów Tumski-Śródka-Zawady-Komandoria",
    "Piątkowo", "Podolany", "Rataje", "Sołacz", "Stare Miasto",
    "Starołęka-Minikowo-Marlewo", "Strzeszyn", "Szczepankowo-Spławie-Krzesinki",
    "Stare Winogrady", "Świerczewo", "Św. Łazarz", "Umultowo", "Wilda",
    "Warszawskie-Pomet-Maltańskie", "Winiary", "Wola", "Żegrze", "Zielony Dębiec"
]

RZE_DISTRICTS = [
    "1000-Lecia", "Baranówka", "Biała", "Budziwój", "Bzianka", "Dąbrowskiego",
    "Drabinianka", "Franciszka Kotuli", "Generała Grota Roweckiego",
    "Generała Władysława Andersa", "Kmity", "Krakowska – Południe",
    "Króla Stanisława Augusta", "Matysówka", "Mieszka I", "Miłocin – św. Huberta",
    "Miłocin", "Nowe Miasto", "Paderewskiego", "Piastów", "Pobitno",
    "Pogwizdów Nowy", "Pułaskiego", "Przybyszówka", "Staromieście",
    "Śródmieście", "Wilkowyja", "Zalesie", "Zawiszy Czarnego", "Zwięczyca"
]

SZC_DISTRICTS = [
    "Arkońskie-Niemierzyn", "Bukowe-Klęskowo", "Bukowo", "Centrum",
    "Dąbie", "Drzetowo-Grabowo", "Głębokie-Pilchowo", "Golęcino-Gocław",
    "Gumieńce", "Kijewo", "Krzekowo-Bezrzecze", "Łękno", "Majowe",
    "Międzyodrze-Wyspa Pucka", "Niebuszewo", "Niebuszewo-Bolinko",
    "Nowe Miasto", "Osów", "Płonia-Śmierdnica-Jezierzyce", "Podjuchy",
    "Pogodno", "Pomorzany", "Skolwin", "Słoneczne", "Stare Miasto",
    "Stołczyn", "Śródmieście-Północ", "Śródmieście-Zachód", "Świerczewo",
    "Turzyn", "Załom", "Zawadzkiego", "Zdroje", "Złocień"
]

TOR_DISTRICTS = [
    "Barbarka", "Bielany", "Bielawy", "Bydgoskie Przedmieście",
    "Chełmińskie Przedmieście", "Czerniewice", "Glinki", "Grębocin nad Strugą",
    "Jakubskie Przedmieście", "Kaszczorek", "Katarzynka", "Koniuchy",
    "Mokre", "Na Skarpie", "Piaski", "Podgórz", "Rubinkowo", "Rudak",
    "Rybaki", "Stare Miasto", "Starotoruńskie Przedmieście", "Stawki",
    "Winnica", "Wrzosy"
]

ZIE_DISTRICTS = [
    "Barcikowice", "Drzonków", "Jany", "Jarogniewice", "Jeleniów", "Kiełpin",
    "Krępa", "Łężyca", "Ługowo", "Nowy Kisielin", "Ochla", "Przylep", "Racula",
    "Raculka", "Sucha", "Zatonie", "Zawada", "Zielona Góra",
    "Zielona Góra – Centrum", "Zielona Góra – Północ", "Zielona Góra – Południe",
    "Zielona Góra – Wschód", "Zielona Góra – Zachód",
    "Zielona Góra – Osiedle Młodych", "Zielona Góra – Osiedle Piastowskie",
    "Zielona Góra – Osiedle Słowiańskie", "Zielona Góra – Osiedle Zawiszy Czarnego",
    "Zielona Góra – Osiedle Wyszyńskiego", "Zielona Góra – Osiedle Wrocławskie"
]

# --- All supported cities ---
CITIES = [
    "Warszawa", "Kraków", "Białystok", "Bydgoszcz", "Gdańsk",
    "Gorzów Wielkopolski", "Kielce", "Katowice", "Lublin", "Łódź",
    "Olsztyn", "Opole", "Poznań", "Szczecin", "Rzeszów",
    "Toruń", "Wrocław", "Zielona Góra"
]

# --- Mapping cities to their districts ---
CITY_TO_DISTRICTS = {
    "Warszawa": WAW_DISTRICTS,
    "Kraków": KRA_DISTRICTS,
    "Białystok": BIA_DISTRICTS,
    "Bydgoszcz": BYD_DISTRICTS,
    "Gdańsk": GDA_DISTRICTS,
    "Gorzów Wielkopolski": GOR_DISTRICTS,
    "Kielce": KIE_DISTRICTS,
    "Katowice": KAT_DISTRICTS,
    "Lublin": LUB_DISTRICTS,
    "Łódź": LOD_DISTRICTS,
    "Olsztyn": OLS_DISTRICTS,
    "Opole": OPO_DISTRICTS,
    "Poznań": POZ_DISTRICTS,
    "Szczecin": SZC_DISTRICTS,
    "Rzeszów": RZE_DISTRICTS,
    "Toruń": TOR_DISTRICTS,
    "Wrocław": WAW_DISTRICTS,  # Example placeholder — update with real districts if needed
    "Zielona Góra": ZIE_DISTRICTS
}

def clean_street_name(name: str) -> str:
    if not isinstance(name, str) or not name.strip():
        return "nieznana"
    name = re.sub(r'^(ul\.|ulica|al\.|aleja)\s+', '', name.strip(), flags=re.I)
    return name.strip()


def parse_localization(text: str) -> pd.Series:
    # Usunięto street
    district = "nieznana"
    city = "nieznane"
    region = "nieznane"

    if not isinstance(text, str) or not text.strip():
        return pd.Series([district, city, region])

    tokens = [t.strip() for t in text.split(',') if t.strip()]
    tokens = [t.replace("–", "-") for t in tokens]

    for token in tokens:
        if token.lower() in REGIONS:
            region = token.lower()
            break

    for token in tokens:
        if token in CITIES:
            city = token
            break

    possible_districts = [t for t in tokens if t not in CITIES and t.lower() not in REGIONS]
    if possible_districts:
        district = possible_districts[-1]

    if city in CITY_TO_DISTRICTS:
        for known_district in CITY_TO_DISTRICTS[city]:
            if district.lower() in known_district.lower() or known_district.lower() in district.lower():
                district = known_district
                break

    if city != "nieznane" and region == "nieznane":
        for voivodeship, cities in REGIONS.items():
            if city in cities:
                region = voivodeship
                break

    if city == "nieznane" and region != "nieznane":
        cities_in_region = REGIONS.get(region.lower(), [])
        if len(cities_in_region) == 1:
            city = f"okolice {cities_in_region[0]}"

    return pd.Series([district, city, region])


def scrape_multiple_cities(urls: dict, limit_per_page: int = 72) -> pd.DataFrame:
    all_data = []

    for city_name, base_url in urls.items():
        print(f"Scraping {city_name}...")

        response = requests.get(base_url.format(1), headers=HEADERS)
        soup = BeautifulSoup(response.content, "html.parser")

        try:
            total_listings_tag = soup.find("span", class_="css-1cwh6ya ehod8gt0")
            total_listings = int(re.sub(r"[^\d]", "", total_listings_tag.get_text(strip=True).split()[-1]))
            pages = (total_listings // limit_per_page) + 1
        except:
            total_listings = "N/A"
            pages = 1
        print(f"{city_name}: total listings = {total_listings}, pages = {pages}")

        progress = widgets.IntProgress(
            value=0,
            min=0,
            max=pages,
            description=f'{city_name}:',
            bar_style='success',
            orientation='horizontal'
        )
        display(progress)

        for page in range(1, pages + 1):
            url = base_url.format(page)
            resp = requests.get(url, headers=HEADERS)
            page_soup = BeautifulSoup(resp.content, "html.parser")
            page_offers = extract_offers_from_page(page_soup)
            all_data.extend(page_offers)

            print(f"{city_name} - page {page}: {len(page_offers)} offers", end="\r")
            progress.value = page
            time.sleep(np.random.uniform(1, 3))

        print(f"\nFinished {city_name}. Total listings collected so far: {len(all_data)}\n")

    df = pd.DataFrame(all_data)

    # --- Walidacja i konwersje ---
    # rooms i floor jako int lub NaN
    for col in ['rooms', 'floor']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int')

    # price jako float, walidacja dużych liczb
    if 'price' in df.columns:
        df['price'] = pd.to_numeric(df['price'].astype(str).str.replace(r"[^\d\.]", "", regex=True), errors='coerce')
        df.loc[df['price'] > 1e7, 'price'] = np.nan  # przykładowa granica: 10 mln

    # area jako float
    if 'area' in df.columns:
        df['area'] = pd.to_numeric(df['area'].astype(str).str.replace(",", ".").str.replace("m²", "").str.replace(" ", ""), errors='coerce')

    # price per m2 tylko jeśli price > 0 i area > 0
    df['price_per_m2'] = np.where(
        (df['price'] > 0) & (df['area'] > 0),
        df['price'] / df['area'],
        np.nan
    )

    # Parse localization
    if 'localization' in df.columns:
        df[['district', 'city', 'region']] = df['localization'].apply(parse_localization)

    return df


def extract_offers_from_page(soup: BeautifulSoup) -> list:
    data = []
    offers = soup.find_all("section", class_="css-ito1if")

    for offer in offers:
        row = {}
        title_tag = offer.find("p", {"data-cy": "listing-item-title"})
        row["title"] = title_tag.get_text(strip=True) if title_tag else None

        loc_tag = offer.find("p", {"data-sentry-component": "Address"})
        row["localization"] = loc_tag.get_text(strip=True) if loc_tag else None

        price_tag = offer.find("span", {"data-sentry-element": "MainPrice"})
        row["price"] = price_tag.get_text(strip=True).replace("\xa0", " ") if price_tag else None

        price_m2_tag = offer.find("span", class_="css-u0t81v")
        row["price_per_m2"] = price_m2_tag.get_text(strip=True).replace("\xa0", " ") if price_m2_tag else None

        seller_wrapper = offer.find("div", {"data-sentry-element": "SellerInfoWrapper"})
        row["type"] = None
        if seller_wrapper:
            type_tag = seller_wrapper.find("span", class_="css-1ig9uyl e11ruw5v4")
            if type_tag:
                seller_type = type_tag.get_text(strip=True).lower()
                if "biuro" in seller_type:
                    row["type"] = "Biuro nieruchomości"
                elif "oferta prywatna" in seller_type:
                    row["type"] = "Oferta prywatna"
                elif "deweloper" in seller_type:
                    continue
                else:
                    row["type"] = seller_type

        dd_tags = offer.find_all("dd", class_="css-17je0kd")
        if len(dd_tags) >= 3:
            rooms_text = dd_tags[0].get_text(strip=True)
            match_rooms = re.search(r'\d+', rooms_text)
            row["rooms"] = int(match_rooms.group()) if match_rooms else None
        
            area_text = dd_tags[1].get_text(strip=True)
            row["area"] = area_text
        
            floor_text = dd_tags[2].get_text(strip=True).lower()
            if "parter" in floor_text:
                row["floor"] = 0
            else:
                match_floor = re.search(r"\d+", floor_text)
                row["floor"] = int(match_floor.group()) if match_floor else None


        data.append(row)
    return data


def scrape_multiple_cities(urls: dict, limit_per_page: int = 72) -> pd.DataFrame:
    """
    urls: dict w formacie {"Miasto": "URL_do_scrapowania z {} dla page"}
    limit_per_page: ile ofert na stronę (zgodne z URL)
    """
    all_data = []

    for city_name, base_url in urls.items():
        print(f"Scraping {city_name}...")

        # Pobierz pierwszą stronę, żeby wykryć liczbę ofert
        response = requests.get(base_url.format(1), headers=HEADERS)
        soup = BeautifulSoup(response.content, "html.parser")

        try:
            total_listings_tag = soup.find("span", class_="css-1cwh6ya ehod8gt0")
            total_listings = int(re.sub(r"[^\d]", "", total_listings_tag.get_text(strip=True).split()[-1]))
            pages = (total_listings // limit_per_page) + 1
        except:
            total_listings = "N/A"
            pages = 1
        print(f"{city_name}: total listings = {total_listings}, pages = {pages}")

        # Widget postępu
        progress = widgets.IntProgress(
            value=0,
            min=0,
            max=pages,
            description=f'{city_name}:',
            bar_style='success',
            orientation='horizontal'
        )
        display(progress)

        for page in range(1, pages + 1):
            url = base_url.format(page)
            resp = requests.get(url, headers=HEADERS)
            page_soup = BeautifulSoup(resp.content, "html.parser")
            page_offers = extract_offers_from_page(page_soup)
            all_data.extend(page_offers)

            print(f"{city_name} - page {page}: {len(page_offers)} offers", end="\r")
            progress.value = page
            time.sleep(np.random.uniform(1, 3))  # losowy delay 1-3s

        print(f"\nFinished {city_name}. Total listings collected so far: {len(all_data)}\n")

    # --- Tworzenie DataFrame ---
    df = pd.DataFrame(all_data)

    # --- Czyszczenie kolumn ---
    if 'price' in df.columns:
        df['price'] = pd.to_numeric(df['price'].astype(str).str.replace(r"[^\d]", "", regex=True), errors='coerce')
    if 'area' in df.columns:
        df['area'] = pd.to_numeric(df['area'].astype(str).str.replace(",", ".").str.replace("m²", "").str.replace(" ", ""), errors='coerce')
    for col in ['rooms', 'floor']:
        if col in df.columns:
            # 1. Konwertuj na numeric i wymuś NaN tam, gdzie nie da się konwertować
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # 2. Konwertuj na nullable Int64
            df[col] = df[col].astype('Int64')
            
    # Price per m2 fallback
    df['price_per_m2'] = (df['price'] / df['area']).round(2)

    # Parse localization
    if 'localization' in df.columns:
        df[['district', 'city', 'region']] = df['localization'].apply(parse_localization)

    return df




# -----------------------------------------------------------------------------
# Run scraper
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    df = scrape_multiple_cities(URLS)

    print("\nScraping done 100%")
    print("Cities found:", df['city'].unique())
    print("Regions found:", df['region'].unique())
    print(f"Saved {len(df)} records.")




Scraping Opole...
Opole: total listings = 267, pages = 4


IntProgress(value=0, bar_style='success', description='Opole:', max=4)

Opole - page 4: 47 offers
Finished Opole. Total listings collected so far: 259

Scraping Toruń...
Toruń: total listings = 905, pages = 13


IntProgress(value=0, bar_style='success', description='Toruń:', max=13)

Toruń - page 13: 27 offers
Finished Toruń. Total listings collected so far: 916

Scraping Bydgoszcz...
Bydgoszcz: total listings = 1750, pages = 25


...

Szczecin - page 30: 23 offers
Finished Szczecin. Total listings collected so far: 19638

Scraping Kraków...
Kraków: total listings = 7192, pages = 100


IntProgress(value=0, bar_style='success', description='Kraków:')

Kraków - page 100: 65 offers
Finished Kraków. Total listings collected so far: 25124

Scraping Warszawa...
Warszawa: total listings = 12323, pages = 172


IntProgress(value=0, bar_style='success', description='Warszawa:', max=172)

Warszawa - page 172: 12 offers
Finished Warszawa. Total listings collected so far: 35956


Scraping done 100%
Cities found: ['Opole' 'okolice Opole' 'Toruń' 'Bydgoszcz' 'nieznane' 'Lublin'
 'okolice Lublin' 'Zielona Góra' 'Łódź' 'okolice Łódź' 'Rzeszów'
 'okolice Rzeszów' 'Białystok' 'okolice Białystok' 'Gdańsk'
 'okolice Gdańsk' 'Katowice' 'okolice Katowice' 'Kielce' 'okolice Kielce'
 'Olsztyn' 'okolice Olsztyn' 'Poznań' 'okolice Poznań' 'Szczecin'
 'okolice Szczecin' 'Kraków' 'okolice Kraków' 'Warszawa'
 'okolice Warszawa']
Regions found: ['opolskie' 'kujawsko-pomorskie' 'lubelskie' 'lubuskie' 'łódzkie'
 'podkarpackie' 'podlaskie' 'pomorskie' 'śląskie' 'świętokrzyskie'
 'warmińsko-mazurskie' 'wielkopolskie' 'zachodniopomorskie' 'małopolskie'
 'mazowieckie']
Saved 35956 records.


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34793 entries, 0 to 34792
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         34793 non-null  object 
 1   localization  34793 non-null  object 
 2   price         34793 non-null  float64
 3   price_per_m2  34610 non-null  float64
 4   type          34793 non-null  object 
 5   rooms         34793 non-null  Int64  
 6   area          34793 non-null  float64
 7   floor         34793 non-null  Int64  
 8   district      34793 non-null  object 
 9   city          34793 non-null  object 
 10  region        34793 non-null  object 
dtypes: Int64(2), float64(3), object(6)
memory usage: 3.0+ MB


In [103]:
print(len(df))

34610


In [99]:
df.isnull().sum()

title             0
localization      0
price             0
price_per_m2    183
type              0
rooms             0
area              0
floor             0
district          0
city              0
region            0
dtype: int64

In [102]:
df.describe().round(2)

,price,price_per_m2,rooms,area,floor
count,34610.00,34610.00,34610.0,34610.00,34610.0
mean,841307.40,14423.34,2.63,58.19,2.68
std,585033.73,5677.54,0.96,24.54,2.42
min,79999.00,5000.00,1.0,11.00,0.0
25%,519000.00,10153.76,2.0,42.10,1.0
50%,690000.00,13483.50,3.0,53.12,2.0
75%,946033.00,17516.45,3.0,67.12,4.0
max,9200000.00,50000.00,10.0,200.00,10.0


In [109]:
df.to_csv("cities_data/otodom_apartments_demo.csv", index=False, encoding="utf-8-sig", sep=";")

In [33]:
print(df['city'].unique(), df['region'].unique())

['Olsztyn' 'okolice Olsztyn'] ['warmińsko-mazurskie']


In [101]:
# --- Filtracja ekstremalnych wartości ---
# Realistyczne zakresy dla kolumn
PRICE_MIN, PRICE_MAX = 1e4, 1e7       # cena w PLN
PRICEM2_MIN, PRICEM2_MAX = 5000, 50000       # cena za metr
AREA_MIN, AREA_MAX = 10, 200          # powierzchnia w m²
ROOMS_MIN, ROOMS_MAX = 1, 10          # liczba pokoi
FLOOR_MIN, FLOOR_MAX = 0, 20          # piętro

# Usuń rekordy z ekstremalnymi wartościami lub zamień na NaN
df['price'] = df['price'].where(df['price'].between(PRICE_MIN, PRICE_MAX), np.nan)
df['area'] = df['area'].where(df['area'].between(AREA_MIN, AREA_MAX), np.nan)
df['rooms'] = df['rooms'].where(df['rooms'].between(ROOMS_MIN, ROOMS_MAX), np.nan)
df['floor'] = df['floor'].where(df['floor'].between(FLOOR_MIN, FLOOR_MAX), np.nan)

# --- Price per m2 ---
df['price_per_m2'] = np.where(
    (df['price'] > 0) & (df['area'] > 0),
    df['price'] / df['area'],
    np.nan
)

df['price_per_m2'] = df['price_per_m2'].where(
    df['price_per_m2'].between(PRICEM2_MIN, PRICEM2_MAX),
    np.nan
)

# Usuwamy rekordy, które mają NaN w price_per_m2
df = df.dropna(subset=['price_per_m2'])

# --- Opcjonalnie: usuń rekordy bez powierzchni lub ceny ---
df = df.dropna(subset=['price', 'area'])

# --- Reset indeksu po filtracji ---
df = df.reset_index(drop=True)

# --- Podsumowanie po oczyszczeniu ---
print("Po oczyszczeniu:")
display(df.describe().round(2))
print("\nBraki w danych po oczyszczeniu:")
display(df.isnull().sum())

Po oczyszczeniu:


,price,price_per_m2,rooms,area,floor
count,34610.00,34610.00,34610.0,34610.00,34610.0
mean,841307.40,14423.34,2.63,58.19,2.68
std,585033.73,5677.54,0.96,24.54,2.42
min,79999.00,5000.00,1.0,11.00,0.0
25%,519000.00,10153.76,2.0,42.10,1.0
50%,690000.00,13483.50,3.0,53.12,2.0
75%,946033.00,17516.45,3.0,67.12,4.0
max,9200000.00,50000.00,10.0,200.00,10.0



Braki w danych po oczyszczeniu:


title           0
localization    0
price           0
price_per_m2    0
type            0
rooms           0
area            0
floor           0
district        0
city            0
region          0
dtype: int64

In [15]:
if (df['city'] == "nieznane").any():
    df['city'] = df['city'].replace("nieznane", "okolice Bydgoszczy")

In [108]:
# Filtrujemy tylko rekordy, które **nie** zawierają 'okolice'
df = df[~df['city'].str.contains('okolice', case=False, na=False)].copy()

# Sprawdzenie
print(df['city'].unique())

['Opole' 'Toruń' 'Bydgoszcz' 'nieznane' 'Lublin' 'Zielona Góra' 'Łódź'
 'Rzeszów' 'Białystok' 'Gdańsk' 'Katowice' 'Kielce' 'Olsztyn' 'Poznań'
 'Szczecin' 'Kraków' 'Warszawa']


In [84]:
df = df.dropna(subset=['floor'])

In [23]:
if (df['district'] == "bydgoski").any():
    df['district'] = df['district'].replace("bydgoski", "inna")

In [42]:
files = [
    "otodom_apartments1.csv",
    "otodom_apartments2.csv",
    "otodom_apartments3.csv"
]

# Wczytanie i połączenie wszystkich CSV w jeden DataFrame
dfs = [pd.read_csv(f, sep=";", encoding="utf-8-sig") for f in files]
df_combined = pd.concat(dfs, ignore_index=True)

print(f"Combined shape: {df_combined.shape}")
df.to_csv("cities_data/otodom_apartments.csv", index=False, sep=";", encoding="utf-8-sig")
print("zapisano")


Combined shape: (40467, 12)
zapisano


In [66]:
df[df['price'] == 7924]

,title,localization,price,price_per_m2,type,rooms,area,floor,district,city,region
85,Opole - Chmielowice mieszkania i domy STRUXI,"Chmielowice, Opole, opolskie",7924.0,NaN,Deweloper,4,NaN,72,Chmielowice,Opole,opolskie
